<a href="https://colab.research.google.com/github/nbchan/INMR96-Digital-Health-and-Data-Analytics/blob/main/Week_07_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

For this tutorial, instead of individual bite-sized exercises, a modified copy of the notebook on Exploratory Data Analysis, Training, Evaluating and Tuning Models (I) would be given. You would need to complete certain parts of the codes yourself (indicated with `## YOUR CODE HERE ##`). For the given parts of the code, try understanding them as much as possible instead of simply mindlessly executing all of them. Refer to the original notebook for solutions.

---

# 1. Setting up

**Packages for data handling and visualization**

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

**Packages for data analysis and modelling**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay, RocCurveDisplay

**Import the intermediate dataset and Authenication for Google Drive access**

If you saved the intermediate dataset in your Google Drive by following all the steps in the previous tutorial, you can grant Colab access to your Google Drive and import the intermediate dataset to the current notebook.

Alternatively, when you are working on your own analysis, you may also put everything (e.g. codes for extracting data from BigQuery; exploring data; preparing and cleaning data; building models) in a single notebook. In that case, you do not need to import the dataset from your Google Drive - simply continue with the cleaned dataset in your existing notebook environment. 


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount = True) # you would need to authenicate yourself here

gdrive_rootpath = '/content/gdrive/MyDrive/' 
# if you saved the file inside a folder in your Google Drive (instead of the base path), 
# you would need to append the folder name to the above

If you have followed the previous tutorial and execute all codes, a file named 'mimic_in_hosp_death_clean.csv' should appear in your Google Drive. In that case you can directly import it into the current notebook. Otherwise, you can download the intermediate dataset using the link below.

In [ ]:
if os.path.exists(gdrive_rootpath + 'mimic_in_hosp_death_clean.csv'):
  print('Importing file from Google Drive...')
  df = pd.read_csv(gdrive_rootpath + 'mimic_in_hosp_death_clean.csv')
else:
  print('Importing file from external link...')
  df = pd.read_csv('https://dl.dropboxusercontent.com/s/0g9rio6jz5zb8ow/mimic_in_hosp_death_clean.csv')

In [ ]:
df.info()

In [ ]:
df.head()

---

# 2. Exploratory Data Analysis (EDA)

The goals of EDA are to:

1. **understand** the data and uncover any problems inherent in the dataset
2. determine if the given dataset is **relevant and sufficient** to answer a specific research question or train a decision support system based on it
3. determine the data **pre-processing or feature engineering** steps required for model training (as discussed in the previous tutorial)
4. **refine** the research problem and/or objectives based on what you have learned about the data

Like many other steps in data analysis, EDA is an iterative process. Here, we will introduce some tools you may use when exploring a dataset. While not demostrated below, one package that may come in handy in EDA is [Pandas Profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/). 

## 2.1. Prediction Target: `IN_HOSP_DEATH`

What is the number of patients who died within the dataset? What is the in-hospital death rate?

In [ ]:
## YOUR CODE HERE ##

---

## 2.2. Categorical features

We can tally and visualise each feature by category using bar charts. You may also apply statistical tests to see if there are significant differences between groups.

For simple visualizations in Python, the package [seaborn](https://seaborn.pydata.org/) may come in handy.


**`GENDER`**

What is the average in-hospital mortality rate by gender?

In [ ]:
## YOUR CODE HERE ##
# hint: use .groupby() followed by .mean()

Visualise it using `sns.barplot()`

In [ ]:
## YOUR CODE HERE ##

What is the average in-hospital mortality rate by each of the following?

* `ADMISSION_TYPE`
* `INSURANCE`

For each feature, also visualise it using `sns.barplot()`.

In [ ]:
## ADMISSION_TYPE
## YOUR CODE HERE ##

In [ ]:
## INSURANCE
## YOUR CODE HERE ##

## 2.3. Numerical features

For univariate analysis, we can calculate basic statistics and visualise them using box plots, histograms and density plots. For multivariate, we can use scatter plots (for two numerical features) and side-by-side histograms (for one numerical and one categorical feature).

**`AGE`**

Calculate some basic statistics for the variable `AGE`.

In [ ]:
## YOUR CODE HERE ##

Visualise the age distribution using `sns.histplot()` and `sns.boxplot()`.

In [ ]:
# histogram
## YOUR CODE HERE ##

In [ ]:
# box plot
## YOUR CODE HERE ##

Plot a box plot to show the age distribution by `IN_HOSP_DEATH`

In [ ]:
# box plot by category
## YOUR CODE HERE ##

In [ ]:
# side-by-side histrograms
g = sns.FacetGrid(df, col = 'IN_HOSP_DEATH', sharey = False, height = 6) # height = 6 specifies the plot size here
g.map(sns.histplot, 'AGE', bins = 20)

**Discussion**: 

* What can you observe from the figure above?
* Based on the figure above, shall we remove the infants (e.g. `AGE` < 1) from our training dataset?

**`DIAG_COUNT`**

Visualise the distribution of `DIAG_COUNT` using `sns.histplot()` and `sns.boxplot()`.

In [ ]:
# histogram
## YOUR CODE HERE ##

In [ ]:
# box plot
## YOUR CODE HERE ##

In [ ]:
g = sns.FacetGrid(df, col = 'IN_HOSP_DEATH', sharey = False, height = 6)
g.map(sns.histplot, 'DIAG_COUNT', bins = 20, kde = True)

As a reference for you, the following code plots the distributions of all numerical features for the two patient groups using a `For` loop.

In [ ]:
numerical_feaures = ['AGE', 'DIAG_COUNT', 'CALLOUT_COUNT_DAY', 'PRES_COUNT_DAY', 
                     'PROC_COUNT_DAY', 'CPT_COUNT_DAY', 'LAB_COUNT_DAY',
                     'INPUTS_CV_COUNT_DAY', 'INPUTS_MV_COUNT_DAY', 'OUTPUT_COUNT_DAY', 
                     'TRANSFER_COUNT_DAY', 'MICRO_COUNT_DAY', 'LOS', 'LOS_ICU']

for col_name in numerical_feaures:
  print(col_name)
  g = sns.FacetGrid(df, col = 'IN_HOSP_DEATH', sharey = False, height = 6)
  g.map(sns.histplot, col_name, bins = 20, kde = True)
  plt.show()
  print('_'*20)

---

# 3. Training a Logistic Regression Model

## 3.1. Train-test Split

* Help **mitigate overfitting** (model learns to fit the noise and error terms in the data) while we train machine learning models
* Allow us to report an **unbiased performance metric** using data unseen by the model
* A 70-30 or 80-20 split is usually the most common. 

![](https://miro.medium.com/max/694/1*tBErXYVvTw2jSUYK7thU2A.png)

([Source](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6))

`IN_HOSP_DEATH` would be our prediction target. Here, let's define a list of 38 column names for features/inputs we would use in the model as not all columns are appropriate (e.g. `SUBJECT_ID`). 

In [ ]:
features_list = ['GENDER_F', 'AGE', 'LOS', 'LOS_ICU', 
                 'CALLOUT_COUNT_DAY', 'PRES_COUNT_DAY', 'PROC_COUNT_DAY',
                 'CPT_COUNT_DAY', 'LAB_COUNT_DAY', 'INPUTS_CV_COUNT_DAY',
                 'INPUTS_MV_COUNT_DAY', 'OUTPUT_COUNT_DAY', 'TRANSFER_COUNT_DAY',
                 'MICRO_COUNT_DAY', 
                 'ADMISSION_TYPE_ELECTIVE', 'ADMISSION_TYPE_EMERGENCY', 'ADMISSION_TYPE_NEWBORN', 'ADMISSION_TYPE_URGENT', 
                 'RELIGION_CATHOLIC', 'RELIGION_NOT SPECIFIED', 'RELIGION_UNOBTAINABLE', 'RELIGION_OTHERS', 
                 'INSURANCE_Medicare', 'INSURANCE_Private', 'INSURANCE_Medicaid', 'INSURANCE_OTHERS',
                 'MARITAL_STATUS_MARRIED', 'MARITAL_STATUS_SINGLE', 'MARITAL_STATUS_UNKNOWN (DEFAULT)', 'MARITAL_STATUS_OTHERS',
                 'LANGUAGE_ENGL', 'LANGUAGE_SPAN', 'LANGUAGE_RUSS', 'LANGUAGE_OTHERS',
                 'ETHNICITY_WHITE', 'ETHNICITY_BLACK/AFRICAN AMERICAN', 'ETHNICITY_UNKNOWN/NOT SPECIFIED', 'ETHNICITY_OTHERS']

len(features_list)

 Then, create a DataFrame (`X`) containing input features only as well as a Series (`y`) containing our prediction target. 

In [ ]:
## YOUR CODE HERE ##

Finally, use `train_test_split()` from the package [Scikit-learn](https://scikit-learn.org/stable/) to perform a *80-20 stratified* train-test split. 

Check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) of the function for examples and usage. Note that the function outputs four things: the input and output datasets for training and testing respectively. 

In [ ]:
## YOUR CODE HERE ##
X_train, X_test, y_train, y_test = 

By using `.shape`, make sure the dimensions of `X_train` and `X_test` are (47180, 38) and (11796, 38) respectively

In [ ]:
## YOUR CODE HERE ##

In [ ]:
## YOUR CODE HERE ##

As a result, our training and test set would include 47,180 and 11,796 admissions respectively. 

## 3.2. Model Training

![](https://static.javatpoint.com/tutorial/machine-learning/images/linear-regression-vs-logistic-regression.png)

([Source](https://www.javatpoint.com/linear-regression-vs-logistic-regression-in-machine-learning))

In this example, we would be utilizing the package package [Scikit-learn](https://scikit-learn.org/stable/user_guide.html) heavily for modelling. In Scikit-learn, the general flow for training and using a model can be simplified into 3 steps:

1. **Specify a model to be trained**
  * Choose a type of model based on your task and dataset. A full list of available models in the package can be found [here](https://scikit-learn.org/stable/modules/classes.html).
  * Set the hyperparameters of the model. 
  * Assign the model to a variable.
2. **Train the model**
  * Call the `.fit()` function using your cleaned train dataset.
3. **Make predictions**
  * Call the `.predict()` function using your cleaned test dataset.

In [ ]:
# 1. Specify a model to be trained
## YOUR CODE HERE ##
# hint: use LogisticRegression()

In [ ]:
# 2. Train the model
## YOUR CODE HERE ##

In [ ]:
# 3. Make predictions (binary)
## YOUR CODE HERE ##

Use `.predict_proba()` instead if you want probabilistic outputs over class outputs. 

In [ ]:
## YOUR CODE HERE ##

---

## 3.3. Evaluation (binary classification)

There are several metrics for evaluating the binary classification performances. In general, the most common reported metric for these kinds of machine learning problems are

* **F1-score**: harmonic mean of precision and recall
* **Area under receiver operating characteristic curve (AUC)**: overall diagnostic ability across all classification probability cutoffs

![binary classification metrics](https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg)

([Source](https://manisha-sirsat.blogspot.com/2019/04/confusion-matrix.html))

<img src='https://upload.wikimedia.org/wikipedia/commons/1/13/Roc_curve.svg' width='40%'>

([Source](https://en.wikipedia.org/wiki/Receiver_operating_characteristic))

In [ ]:
# to calculate accuracy:
## YOUR CODE HERE ##
# hint: use `accuracy_score()`

In [ ]:
# to calculate F1 score:
## YOUR CODE HERE ##
# hint: use `f1_score()`

In [ ]:
# to display multiple classification-related metrics:
print(classification_report(y_test, y_pred_lr, digits = 3))

In [ ]:
# to visualise a confusion matrix using a heatmap:
ConfusionMatrixDisplay.from_estimator(model_lr, X_test, y_test)
plt.grid(False) # just to disable grid lines for this plot

Note that the colour in the above plot has little implications as the quantity of the two classes differ by too much. We can express them as ratios and percentages instead. 

In [ ]:
# recall-related metrics (% of row)
ConfusionMatrixDisplay.from_estimator(model_lr, X_test, y_test, normalize = ## YOUR CODE HERE ##) 
plt.grid(False)

In [ ]:
# precision-related metrics (% of column)
ConfusionMatrixDisplay.from_estimator(model_lr, X_test, y_test, normalize = ## YOUR CODE HERE ##)
plt.grid(False)

In [ ]:
# to plot an ROC:
fig, ax = plt.subplots(figsize=(6, 6)) # modify figure size
RocCurveDisplay.from_estimator(model_lr, X_test, y_test, ax = ax)

In [ ]:
# to calculate Area under ROC:
## YOUR CODE HERE ##
# hint: use `roc_auc_score()`
# hint: use probabilistic predictions over class predictions

**Some Observations**

* Despite having an accuracy of 92.6%, the overall F1-score is around 48%. The high accuracy is due to class imbalance and the model predicted the majority class (`IN_HOSP_DEATH == 0`) correctly. 
* The recall for the positive examples is 34%, meaning that only around one-third of those who died within their hospital stay is successfully detected by the model.

---

# 4. Training a Random Forest Model

## 4.1. Train-test Split

As we have already made a train-test split when training the logistic regression model, we do not need to do this again. Having the same data split also allows for fair comparison between models.

---

## 4.2. Model Training

* A Random Forest model consists of **multiple Decision Tree** models.
* Each decision tree consists of **levels** of nodes that represent **yes/no questions**. By following these rules given by a decision tree, each sample arrives to the end node which outputs a prediction. 
* Decision trees are trained by minimizing metrics such as gini score or **entropy**. 
* Random Forest is trained by first training multiple decision trees with different subsets of the training dataset as inputs; then **aggregate the predictions from all decision trees** via a majority vote or averaging. 

![Random Forest](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/02/rfc_vs_dt1.png)

([Source](https://www.analyticsvidhya.com/blog/2020/05/decision-tree-vs-random-forest-algorithm/))

We can follow the same 3-step process to train the model.

Define the configuration/hyperparameters of a random forest model. Here we specify that the model contains 200 decision trees.

In [ ]:
# 1. Specify a model to be trained
# Also define the configuration/hyperparameters of a random forest model. 
# Here we specify that the model contains 200 decision trees.

## YOUR CODE HERE ##
# hint: use RandomForestClassifier()

In [ ]:
# 2. Train the model
## YOUR CODE HERE ##

In [ ]:
# 3. Make predictions
## YOUR CODE HERE ##

Use `.predict_proba()` instead if you want probabilistic outputs over class outputs.

In [ ]:
## YOUR CODE HERE ##

---

## 4.3. Evaluation

In [ ]:
# to calculate accuracy:
## YOUR CODE HERE ##

In [ ]:
# to calculate F1 score:
## YOUR CODE HERE ##

In [ ]:
# to display multiple classification-related metrics:
print(classification_report(y_test, y_pred_rf, digits = 3))

In [ ]:
# to visualise a confusion matrix using a heatmap:
# recall-related metrics (% of row)
ConfusionMatrixDisplay.from_estimator(model_rf, X_test, y_test, normalize = 'true') 
plt.grid(False)

In [ ]:
# precision-related metrics (% of column)
ConfusionMatrixDisplay.from_estimator(model_rf, X_test, y_test, normalize = 'pred')
plt.grid(False)

In [ ]:
# to plot an ROC:
fig, ax = plt.subplots(figsize=(6, 6)) # modify figure size
RocCurveDisplay.from_estimator(model_rf, X_test, y_test, ax = ax)

In [ ]:
# to calculate Area under ROC:
## YOUR CODE HERE ##
# hint: use probabilistic predictions over class predictions

Let us compare the performance of the two models with the table below: 

In [ ]:
pd.DataFrame({
    'Random Forest': [
        accuracy_score(y_test, y_pred_rf), 
        f1_score(y_test, y_pred_rf), 
        precision_score(y_test, y_pred_rf), 
        recall_score(y_test, y_pred_rf), 
        roc_auc_score(y_test, y_pred_prob_rf[:,1])
    ], 
    'Logistic Regression': [
        accuracy_score(y_test, y_pred_lr), 
        f1_score(y_test, y_pred_lr), 
        precision_score(y_test, y_pred_lr), 
        recall_score(y_test, y_pred_lr), 
        roc_auc_score(y_test, y_pred_prob_lr[:,1])
    ]
    }, index=[
        'Accuracy', 'F1 Score', 'Precision', 'Recall', 'ROC'
    ])

**Some Observations**

* This model outperforms the logistic regression model we have trained last tutorial in several metrics:
* The recall is still not optimal at 45.5%, meaning that only around half of those who died within their hospital stay is successfully detected by the model. Depending on the objectives of your model, it **could be beneficial to choose another cutoff point** for binary classification predictions rather than the default 0.5. One might prioritise recall over precision if the impact of having false negatives outweighs false positives. For example, for cancer prediction, minimising the risk of unidentified positive cases (hence delayed treatment) could outweigh the inconvenience and costs incurred by additional checkup appointments for negative cases. 

### Tuning Cutoffs

As an example, let's lower the cutoff to 0.2 so that more patients will be classified as positive (hence higher recall at the cost of lower precision). We can use the probablistic outputs from the model (saved in `y_pred_prob_rf`) and define a new cutoff ourselves. 

Plot a histogram of the probabilistic predictions of the random forest model.

In [ ]:
## YOUR CODE HERE ##
# hint: use probabilistic predictions over class predictions

Lower the prediction cutoff to 0.2 and save it as a new variable.

In [ ]:
## YOUR CODE HERE ##

Count the number of predicted positives based on cutoffs 0.5 (the default) and 0.2.

In [ ]:
## YOUR CODE HERE ##

In [ ]:
## YOUR CODE HERE ##

In [ ]:
# to display multiple classification-related metrics:
print(classification_report(y_test, y_pred_rf_cutoff20, digits = 3))

In [ ]:
# recall-related metrics (% of row)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf_cutoff20, normalize = 'true') 
plt.grid(False)

In [ ]:
# precision-related metrics (% of column)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf_cutoff20, normalize = 'pred')
plt.grid(False)

* As we lower the cutoff, more patients were classified as positive (613 -> 1,680), leading to a better recall (45.5% -> 75.7%) and a worse precision (84.7% -> 52.7%)

---

## 4.4. Understanding a Random Forest Model

One major strength of tree-based machine learning models is its inherent ability to easily report **feature importance**. This could be useful to finding insights in the model.

In [ ]:
model_rf_importance = pd.Series(## YOUR CODE HERE ##
                                , index = features_list).sort_values(ascending=False)
model_rf_importance

In [ ]:
plt.figure(figsize = (10, 10))
sns.barplot(x = model_rf_importance, y = model_rf_importance.index)

## 4.6. Saving and Loading your trained model

After training a model you might want to save your model to your Google Drive so that you do not need to re-train it next time. (As a side note, due to the random nature of most machine learning algorithms, if you did not set a random seed (`random_state`) while training, you would end up with a model with different trained parameters when you re-train a model)

In [ ]:
import joblib

To save:

In [ ]:
joblib.dump(model_rf, gdrive_rootpath + 'mimic_in_hosp_mortality_model_rf.joblib') # specify your filename here

To load:

In [ ]:
model_rf_loaded = joblib.load(gdrive_rootpath + 'mimic_in_hosp_mortality_model_rf.joblib') # specify your filename here

In [ ]:
model_rf_loaded

## 4.7. Discussion

How well will the models above translate into a decision support system in practice? Here are some issues you may think about (which is beyond the scope of training machine learning models itself):

* What are the purposes or goals of the model? Where would it lie on the patient pathway? Do they align with the interests of clinicians, patients or other stakeholders? 
* Suppose we trained a reasonably reliable model. What could be the associated interventions based on the model predictions?
* What are the limitations of the model from an operational perspective? 
  * e.g. consider the time of which the variables becomes available. Could the model deliver the predictions in a timely manner? Can this be improved? 

---

# Extras

You may also explore the [COVID-19 worldwide dataset](https://github.com/owid/covid-19-data/blob/master/public/data/) by comparing statistics on Jan 1, 2022 across `continent`s. Here are some codes to help you get started.

In [ ]:
df_covid = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
df_covid_filtered = df_covid[df_covid['date'] == '2022-01-01'] # filter by date
df_covid_filtered = df_covid_filtered[~df_covid_filtered['iso_code'].str.startswith('OWID_')] # removes non-country locations such as "Worldwide", "Asia" and "Low income"

df_covid_filtered

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
1041,AFG,Asia,Afghanistan,2022-12-31,207559.0,9.0,35.571,7849.0,2.0,0.571,...,NaN,37.746,0.50,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3251,ALB,Europe,Albania,2022-12-31,333806.0,0.0,8.143,3595.0,0.0,0.000,...,51.2,NaN,2.89,78.57,0.795,2842318.0,NaN,NaN,NaN,NaN
4350,DZA,Africa,Algeria,2022-12-31,271228.0,5.0,4.857,6881.0,0.0,0.000,...,30.4,83.741,1.90,76.88,0.748,44903228.0,NaN,NaN,NaN,NaN
5443,AND,Europe,Andorra,2022-12-31,47751.0,0.0,9.286,165.0,0.0,0.000,...,37.8,NaN,NaN,83.73,0.868,79843.0,NaN,NaN,NaN,NaN
6518,AGO,Africa,Angola,2022-12-31,105095.0,0.0,17.429,1930.0,0.0,0.286,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254239,VNM,Asia,Vietnam,2022-12-31,11525231.0,87.0,147.000,43186.0,0.0,0.286,...,45.9,85.847,2.60,75.40,0.704,98186856.0,NaN,NaN,NaN,NaN
256168,WLF,Oceania,Wallis and Futuna,2022-12-31,3415.0,0.0,0.000,7.0,0.0,0.000,...,NaN,NaN,NaN,79.94,NaN,11596.0,NaN,NaN,NaN,NaN
258356,YEM,Asia,Yemen,2022-12-31,11945.0,0.0,0.000,2159.0,0.0,0.000,...,29.2,49.542,0.70,66.12,0.470,33696612.0,NaN,NaN,NaN,NaN
259433,ZMB,Africa,Zambia,2022-12-31,334425.0,0.0,57.714,4024.0,0.0,0.714,...,24.7,13.938,2.00,63.89,0.584,20017670.0,NaN,NaN,NaN,NaN


# References

* [Which machine learning algorithm should I use?](https://blogs.sas.com/content/subconsciousmusings/2020/12/09/machine-learning-algorithm-use/)
![](https://blogs.sas.com/content/subconsciousmusings/files/2017/04/machine-learning-cheet-sheet-2.png)
* [Decision Tree vs. Random Forest – Which Algorithm Should you Use?](https://www.analyticsvidhya.com/blog/2020/05/decision-tree-vs-random-forest-algorithm/)
* [Train/Test Split and Cross Validation in Python](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)
